##### LAB 3.03: Using MCP in LangChain
Dina Bosma-Buczynska

**Step 1: Setup and Installation**

In [1]:
import nest_asyncio
import asyncio
import sniffio

nest_asyncio.apply()

# ── Python 3.14 compatibility patches ──────────────────────────────────────
#
# Python 3.14 changed how asyncio tracks the "current task", breaking two
# assumptions made by anyio (and libraries that depend on it: mcp, httpcore):
#
#   1. sniffio can't detect asyncio via asyncio._get_running_loop() → patched
#      so it falls back to "asyncio" instead of raising AsyncLibraryNotFoundError.
#
#   2. anyio's CancelScope uses a WeakKeyDictionary keyed by asyncio.Task.
#      In Python 3.14 + nest_asyncio, asyncio.current_task() returns None
#      inside create_task() coroutines, causing a TypeError (WeakKeyDictionary
#      can't hold None) that is NOT caught by the existing `except KeyError`.
#      Both __enter__ and __exit__ are patched to handle host_task=None.

# Patch 1 – sniffio
_orig_detect = sniffio.current_async_library

def _detect_with_asyncio_fallback():
    try:
        return _orig_detect()
    except sniffio.AsyncLibraryNotFoundError:
        return "asyncio"

sniffio.current_async_library = _detect_with_asyncio_fallback

# Patch 2 – anyio CancelScope
from anyio._backends import _asyncio as _anyio_asyncio

_orig_cs_enter = _anyio_asyncio.CancelScope.__enter__
_orig_cs_exit  = _anyio_asyncio.CancelScope.__exit__

def _patched_cs_enter(self):
    if asyncio.current_task() is None:
        # Minimal scope setup – skip WeakKeyDictionary operations that require
        # a non-None key.
        if self._active:
            raise RuntimeError(
                "Each CancelScope may only be used for a single 'with' block"
            )
        self._host_task = None
        self._timeout()
        self._active = True
        if self._cancel_called:
            self._deliver_cancellation(self)
        return self
    return _orig_cs_enter(self)

def _patched_cs_exit(self, exc_type, exc_val, exc_tb):
    if self._host_task is None:
        # Mirror of patched __enter__ – clean up without touching _task_states.
        if not self._active:
            raise RuntimeError("This cancel scope is not active")
        self._active = False
        if self._timeout_handle:
            self._timeout_handle.cancel()
            self._timeout_handle = None
        return False
    return _orig_cs_exit(self, exc_type, exc_val, exc_tb)

_anyio_asyncio.CancelScope.__enter__ = _patched_cs_enter
_anyio_asyncio.CancelScope.__exit__  = _patched_cs_exit

print("nest_asyncio applied.")
print("Python 3.14 compatibility patches applied (sniffio + anyio CancelScope).")

nest_asyncio applied.
Python 3.14 compatibility patches applied (sniffio + anyio CancelScope).


In [2]:
# Install all required packages
# Run this once. The -q flag keeps output quiet.
!pip install langchain langchain-openai langchain-mcp-adapters langgraph mcp python-dotenv -q

In [3]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# Load environment variables from .env file
load_dotenv()

# Verify API key is loaded
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please create a .env file with your API key.")

# Set up the language model
# temperature=0 means the model gives consistent, predictable answers
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("OpenAI API key loaded successfully.")
print("LLM (gpt-4o-mini) is ready.")

OpenAI API key loaded successfully.
LLM (gpt-4o-mini) is ready.


**Step 2: Connect to MCP Server**

In [4]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import asyncio

# Configure the LangChain documentation MCP server
# MultiServerMCPClient takes a dict of server configs
# Each config specifies transport type and connection details
mcp_client = MultiServerMCPClient({
    "langchain-docs": {
        "transport": "http",
        "url": "https://docs.langchain.com/mcp"
    }
})

print("MCP client configured for LangChain documentation server")
print(f"Server: langchain-docs -> https://docs.langchain.com/mcp")

MCP client configured for LangChain documentation server
Server: langchain-docs -> https://docs.langchain.com/mcp


**Step 3: Load MCP Tools into LangChain**

In [5]:
# Load tools from the MCP server
# The client is stateless: get_tools() creates ephemeral sessions under the hood
mcp_tools = await mcp_client.get_tools()

print(f"Loaded {len(mcp_tools)} tools from MCP server(s)")
print("\nAvailable tools:")
for tool in mcp_tools:
    print(f"  - {tool.name}: {tool.description[:80]}...")

Loaded 1 tools from MCP server(s)

Available tools:
  - SearchDocsByLangChain: Search across the Docs by LangChain knowledge base to find relevant information,...


**Step 4: Create Agent with MCP Tools**

In [6]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

# Get tools from MCP server
tools = await mcp_client.get_tools()

# Create agent with the model and MCP tools
# Use the 'prompt' parameter to set the system message
agent = create_react_agent(
    model=llm,
    tools=tools,
    prompt="You are a helpful assistant that can answer questions about LangChain, LangGraph, and LangSmith by searching the official documentation. Always provide accurate, up-to-date information based on the documentation."
)

print(f"Agent created with {len(tools)} MCP tools")
print(f"Tools: {[tool.name for tool in tools]}")

Agent created with 1 MCP tools
Tools: ['SearchDocsByLangChain']


/var/folders/gh/r4_2cb497nl1c31dzl76npj80000gp/T/ipykernel_5549/2627320630.py:9: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


**Step 5: Access MCP Resources**

In [7]:
from mcp.shared.exceptions import McpError

# Step 5: Access MCP Resources
# Resources are read-only background data — different from tools (which perform actions)

# 1. List available resources using get_resources()
print("=== 1. Listing Available MCP Resources ===\n")

mcp_resources = []
try:
    mcp_resources = await mcp_client.get_resources()
except* McpError as eg:
    # anyio's TaskGroup wraps the McpError in an ExceptionGroup.
    # This server returns "Method not found" because it only exposes tools.
    for exc in eg.exceptions:
        print(f"[INFO] get_resources() not supported by this server: {exc}")

print(f"Loaded {len(mcp_resources)} resource(s) from MCP server(s)")

if mcp_resources:
    print("\nAvailable resources:")
    for resource in mcp_resources:
        print(f"  - URI:         {resource.uri}")
        print(f"    Name:        {resource.name}")
        print(f"    Description: {getattr(resource, 'description', 'N/A')}")
        print()

    # 2. Read resource content using read_resource()
    print("=== 2. Reading Resource Content ===\n")
    first_uri = mcp_resources[0].uri
    resource_content = await mcp_client.read_resource(first_uri)

    print(f"Resource URI: {first_uri}")
    print(f"Content preview (first 400 chars):\n{str(resource_content)[:400]}...\n")

    # 3. Use resource content in agent context
    print("=== 3. Using Resource as Agent Context ===\n")

    context_question = f"""Using the following background context from the LangChain documentation:

{str(resource_content)[:1000]}

Based on this context: What is LangChain and what are its main use cases?"""

    result = await agent.ainvoke({
        "messages": [HumanMessage(content=context_question)]
    })

    print(f"Answer (with resource context):\n{result['messages'][-1].content}")

else:
    # Server does not expose resources — fall back to a standard agent query
    print("\nThis server only exposes tools, not resources.")
    print("Running a standard agent query for demonstration...\n")

    question = "What is LangChain and what are its main use cases?"
    result = await agent.ainvoke({
        "messages": [HumanMessage(content=question)]
    })

    print(f"Answer: {result['messages'][-1].content}")

=== 1. Listing Available MCP Resources ===

[INFO] get_resources() not supported by this server: unhandled errors in a TaskGroup (1 sub-exception)
Loaded 0 resource(s) from MCP server(s)

This server only exposes tools, not resources.
Running a standard agent query for demonstration...

Answer: LangChain is an open-source framework designed to facilitate the development of applications powered by large language models (LLMs). It provides a pre-built agent architecture and integrations with various models and tools, allowing developers to create custom agents and applications with minimal code—often under 10 lines.

### Main Use Cases of LangChain:
1. **Building Custom Agents**: LangChain allows developers to create agents that can interact with various LLMs and tools, adapting to the evolving ecosystem of AI technologies.
2. **Integration with Multiple Models**: It supports integration with a wide range of models, including those from OpenAI, Anthropic, Google, and more, enabling seaml

---
##### Debugging Log — Issues Encountered in Step 5

---

Issue 1 — `McpError: Method not found` on `get_resources()`

**What happened:**
`await mcp_client.get_resources()` crashed with a nested `ExceptionGroup` containing `McpError: Method not found`.

**Why:**
The `langchain-docs` server only exposes tools, not resources. It has no `list_resources` handler, so the MCP protocol returned an error code for an unsupported method. anyio's `TaskGroup` then wrapped that error in an `ExceptionGroup`.

**Fix:**
Wrapped the call in `try / except* McpError` (Python 3.11 ExceptionGroup syntax). When the error is caught, `mcp_resources` stays as `[]` and the cell falls through to the `else` branch, which runs a standard agent query instead.

---

Issue 2 — `SyntaxError: cannot have both 'except' and 'except*'`

**What happened:**
The initial fix used both `except McpError` and `except* McpError` in the same `try` block, causing a `SyntaxError`.

**Why:**
Python 3.11 does not allow mixing classic `except` and `except*` (ExceptionGroup) clauses in the same `try` statement.

**Fix:**
Removed the plain `except McpError` clause and kept only `except* McpError`, since anyio always wraps task exceptions in an `ExceptionGroup`.

**Step 6: Build a Complete MCP-Enabled Agent**

In [8]:
import os, sys
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

server_script = os.path.join(os.path.abspath("."), "local_mcp_server.py")

# ── 1. Set up MultiServerMCPClient with TWO servers ───────────────────────
#
#   langchain-docs  → HTTP, remote  → exposes: Tools only
#   local-resources → stdio, local  → exposes: Resources only
#
mcp_client_full = MultiServerMCPClient({
    "langchain-docs": {
        "transport": "http",
        "url": "https://docs.langchain.com/mcp",
    },
    "local-resources": {
        "transport": "stdio",
        "command": sys.executable,
        "args": [server_script],
    },
})

print("=== 1. MCP Client Configured ===")
print(f"Servers: {list(mcp_client_full.connections.keys())}\n")

# ── 2. Load tools from all servers ────────────────────────────────────────
tools = await mcp_client_full.get_tools()

print("=== 2. Tools Loaded ===")
print(f"Total tools: {len(tools)}")
for t in tools:
    print(f"  - {t.name}: {t.description[:70]}...")

# ── 3. Load resources from local server only ──────────────────────────────
# langchain-docs raises "Method not found" for list_resources.
# asyncio.gather() fails if any server fails, so we target the local server
# with a dedicated single-server client.
#
# get_resources() returns Blob objects (langchain_core) — not raw MCP Resource
# objects. Each Blob has:
#   .source   → the resource URI string
#   .data     → the full text content (already fetched)
#   .mimetype → MIME type

mcp_client_local = MultiServerMCPClient({
    "local-resources": {
        "transport": "stdio",
        "command": sys.executable,
        "args": [server_script],
    }
})

resources = await mcp_client_local.get_resources()

print(f"\n=== 3. Resources Loaded ===")
print(f"Total resources (from local server): {len(resources)}")
for r in resources:
    print(f"  - source:  {r.source}")
    print(f"    mime:    {r.mimetype}")
    print(f"    preview: {str(r.data)[:80].strip()}...")
    print()

# ── 4. Use resource content as agent context ──────────────────────────────
# .data already contains the full text — no separate read_resource() needed.
print("=== 4. Resource Content (first resource) ===")
first_blob = resources[0]
content_text = first_blob.data if isinstance(first_blob.data, str) else first_blob.data.decode()
print(f"Source: {first_blob.source}")
print(f"\nContent preview:\n{content_text[:400]}...")

# ── 5. Build comprehensive agent ──────────────────────────────────────────
system_prompt = (
    "You are an expert assistant for the LangChain ecosystem. "
    "You have real-time access to official LangChain docs via MCP tools, "
    "and to AI concept reference material via MCP resources.\n\n"
    "Guidelines:\n"
    "- Search the docs for up-to-date API information.\n"
    "- Use any resource context provided in the user message.\n"
    "- Include concrete code examples when they help.\n"
    "- After answering, suggest one related topic to explore next."
)

comprehensive_agent = create_react_agent(
    model=llm,
    tools=tools,
    prompt=system_prompt,
)

print(f"\n=== 5. Comprehensive Agent Ready ===")
print(f"Tools:     {[t.name for t in tools]}")
print(f"Resources: {len(resources)} loaded\n")

# ── 6. Test with real-world scenarios ─────────────────────────────────────
resource_context = f"Background context from MCP resource ({first_blob.source}):\n{content_text[:800]}\n\n"

test_queries = [
    ("resource + tool", resource_context + "Based on the context above and the docs: what is RAG and how do I implement it in LangChain?"),
    ("tool only",       "What is LangGraph and how does it differ from regular LangChain chains?"),
    ("tool only",       "What is LangSmith and how do I use it to debug a slow agent?"),
]

for i, (mode, question) in enumerate(test_queries, 1):
    print(f"{'='*60}")
    print(f"Query {i} [{mode}]")
    print('='*60)
    result = await comprehensive_agent.ainvoke({
        "messages": [HumanMessage(content=question)]
    })
    print(result["messages"][-1].content)
    print()

# ── 7. Cleanup ─────────────────────────────────────────────────────────────
del mcp_client_full, mcp_client_local
print("=== 7. Cleanup Complete ===")
print("Both MCP clients released.")

=== 1. MCP Client Configured ===
Servers: ['langchain-docs', 'local-resources']

=== 2. Tools Loaded ===
Total tools: 1
  - SearchDocsByLangChain: Search across the Docs by LangChain knowledge base to find relevant in...

=== 3. Resources Loaded ===
Total resources (from local server): 3
  - source:  None
    mime:    text/plain
    preview: Retrieval-Augmented Generation (RAG)
RAG e...

  - source:  None
    mime:    text/plain
    preview: AI Agents
An agent lets an LLM decide which tools to call, in what ord...

  - source:  None
    mime:    text/plain
    preview: Model Context Protocol (MCP)
MCP is an open standa...

=== 4. Resource Content (first resource) ===
Source: None

Content preview:
Retrieval-Augmented Generation (RAG)
RAG enhances LLM responses by retrieving relevant context from a knowledge base
before generating an answer, reducing hallucinations and grounding answers in data.

Key components:
  1. Document Loader  – ingests source documents (PDFs, web pages, database

/var/folders/gh/r4_2cb497nl1c31dzl76npj80000gp/T/ipykernel_5549/824590098.py:85: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  comprehensive_agent = create_react_agent(


Retrieval-Augmented Generation (RAG) is a technique that enhances the capabilities of Large Language Models (LLMs) by retrieving relevant external knowledge before generating responses. This approach helps reduce hallucinations and ensures that the answers are grounded in actual data. The RAG process typically involves two main steps:

1. **Retrieve**: Given a user input, relevant document chunks are retrieved from a storage system using a retriever.
2. **Generate**: The LLM generates an answer using both the user query and the retrieved context.

### Implementing RAG in LangChain

To implement RAG in LangChain, you can follow these steps:

1. **Document Loader**: Load your source documents (e.g., PDFs, web pages).
2. **Text Splitter**: Split the documents into manageable chunks.
3. **Embeddings**: Convert the text chunks into dense vectors for similarity search.
4. **Vector Store**: Store the embeddings in a vector store (e.g., FAISS, Chroma).
5. **Retriever**: Use a retriever to find

---
##### Debugging Log — Issues Encountered in Step 6

---

Issue 1 — `McpError: Method not found` on `get_resources()` (multi-server client)

**Why:** `asyncio.gather()` calls all servers in parallel. When `langchain-docs` fails on `list_resources`, the gather fails and drops results from every other server too.

**Fix:** Load resources using a dedicated single-server client (`mcp_client_local`) pointing only at the local server, so one failing server can never kill the gather for a working one.

---

Issue 2 — `SyntaxError: cannot have both 'except' and 'except*'`

**Why:** Python 3.11 forbids mixing classic `except` and `except*` (ExceptionGroup) in the same `try` block.

**Fix:** Keep only `except* McpError` — anyio's `TaskGroup` always wraps exceptions in an `ExceptionGroup`.

---

Issue 3 — `McpError: Method not found` on `get_tools()` for the local server

**Why:** The local server had no `@app.list_tools()` handler. Every MCP server must respond to every standard method, even with an empty list.

**Fix:** Added `@app.list_tools()` returning `[]` to `local_mcp_server.py`.

---

Issue 4 — `RuntimeError: Error fetching resource local://ai-concepts/rag`

**Why:** The `@app.read_resource()` handler returned `[TextContent(...)]` — a protocol-level object. The mcp server API expects a plain `str` or `bytes`; the library handles protocol wrapping internally.

**Fix:** Changed the handler to return the raw string. Also added `.rstrip("/")` on the URI, as Pydantic's `AnyUrl` can append a trailing slash to custom schemes.

---

Issue 5 — `AttributeError: 'Blob' object has no attribute 'name'`

**Why:** `get_resources()` does not return MCP `Resource` objects — it fetches content immediately and returns `Blob` objects from `langchain_core`, which have different attributes.

| Expected | Actual (`Blob`) |
|---|---|
| `r.name` / `r.uri` / `r.description` | ❌ |
| — | `r.source` · `r.data` · `r.mimetype` ✅ |

**Fix:** Updated all resource access to use `.source`, `.data`, `.mimetype`. Since `.data` already holds the full content, the separate `read_resource()` call was also removed.

---
**Step 6b: Adding a Full-Featured MCP Server (Tools + Prompts)**

The existing `local_mcp_server.py` exposes **Resources only**.  
Here we add a second local server — `local_mcp_server_full.py` — that exposes  
**Tools** and **Prompts**, demonstrating the third MCP capability not yet covered.

The tools are LLM-dev utilities — tasks commonly needed when building LangChain apps:

| Tool | What it does |
|---|---|
| `estimate_tokens` | Approximate GPT token count for a text (~4 chars/token rule) |
| `chunk_text` | Split text into overlapping character chunks (RAG prep) |
| `format_prompt` | Fill a `{variable}` prompt template with values |

**Updated three-server architecture:**

| Server | Transport | Capabilities |
|---|---|---|
| `langchain-docs` | HTTP (remote) | Tools only (docs search) |
| `local-resources` | stdio (local) | Resources only (AI concept notes) |
| `local-full` | stdio (local) | **Tools + Prompts** ← NEW |

> **MCP Prompts** are reusable, server-side prompt templates the client can list,  
> retrieve, and render with arguments — like a function that returns a formatted prompt string.

In [ ]:
import os, sys

server_full_script = os.path.join(os.path.abspath("."), "local_mcp_server_full.py")

assert os.path.exists(server_full_script), (
    f"Server file not found: {server_full_script}\n"
    "Make sure local_mcp_server_full.py is in the same directory as this notebook."
)

print("local_mcp_server_full.py found.")
print(f"Path: {server_full_script}")
print("\nTools exposed by this server:")
print("  estimate_tokens(text)                         → approx GPT token count")
print("  chunk_text(text, chunk_size=200, overlap=20)  → overlapping char chunks")
print("  format_prompt(template, variables)            → fill {placeholder} template")
print("\nPrompts exposed by this server:")
print("  summarise_topic(topic, length?)")
print("  explain_concept(concept)")
print("  compare_tools(tool_a, tool_b)")

---
**Step 5 (Revisited): Tools + Resources Across Three Servers**

Re-running Step 5 with the three-server client to verify the new server's  
tools load correctly alongside the existing remote and resource servers.

In [16]:
import sys, os
from langchain_mcp_adapters.client import MultiServerMCPClient

server_script      = os.path.join(os.path.abspath("."), "local_mcp_server.py")
server_full_script = os.path.join(os.path.abspath("."), "local_mcp_server_full.py")

# ── Three-server client ────────────────────────────────────────────────────
mcp_client_v2 = MultiServerMCPClient({
    "langchain-docs": {
        "transport": "http",
        "url": "https://docs.langchain.com/mcp",
    },
    "local-resources": {
        "transport": "stdio",
        "command": sys.executable,
        "args": [server_script],
    },
    "local-full": {
        "transport": "stdio",
        "command": sys.executable,
        "args": [server_full_script],
    },
})

print("=== Three-Server Client Configured ===")
print(f"Servers: {list(mcp_client_v2.connections.keys())}\n")

# ── Load tools from all three servers ─────────────────────────────────────
tools_v2 = await mcp_client_v2.get_tools()
print(f"=== Tools Loaded ({len(tools_v2)} total) ===")
for t in tools_v2:
    print(f"  - {t.name}: {t.description[:70]}")

# ── Load resources (local-resources server only) ───────────────────────────
# Dedicated single-server client avoids gather failures from other servers.
mcp_client_res_v2 = MultiServerMCPClient({
    "local-resources": {
        "transport": "stdio",
        "command": sys.executable,
        "args": [server_script],
    },
})
resources_v2 = await mcp_client_res_v2.get_resources()

print(f"\n=== Resources Loaded ({len(resources_v2)} total) ===")
for r in resources_v2:
    print(f"  - {r.source}")
    print(f"    Preview: {str(r.data)[:70].strip()}...")

del mcp_client_v2, mcp_client_res_v2

=== Three-Server Client Configured ===
Servers: ['langchain-docs', 'local-resources', 'local-full']

=== Tools Loaded (4 total) ===
  - SearchDocsByLangChain: Search across the Docs by LangChain knowledge base to find relevant in
  - estimate_tokens: Estimate the number of GPT tokens in a text string. Uses the ~4 chars/
  - chunk_text: Split a text into overlapping chunks by character count. Useful for RA
  - format_prompt: Fill a prompt template that uses {variable} placeholders. Pass the tem

=== Resources Loaded (3 total) ===
  - None
    Preview: Retrieval-Augmented Generation (RAG)
=================================...
  - None
    Preview: AI Agents
An agent lets an LLM decide which tools to call, i...
  - None
    Preview: Model Context Protocol (MCP)
MCP is an o...


---
**Step 6 (Revisited): Full Three-Server Agent + MCP Prompts**

Using all three servers. This step also demonstrates **`get_prompts()`** —  
fetching prompt templates from the `local-full` server, rendering them with  
arguments, and using the result as structured input to the agent.

In [15]:
import os, sys
from mcp.shared.exceptions import McpError
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

server_script      = os.path.join(os.path.abspath("."), "local_mcp_server.py")
server_full_script = os.path.join(os.path.abspath("."), "local_mcp_server_full.py")

# ── 1. Three-server client ─────────────────────────────────────────────────
mcp_client_3 = MultiServerMCPClient({
    "langchain-docs": {
        "transport": "http",
        "url": "https://docs.langchain.com/mcp",
    },
    "local-resources": {
        "transport": "stdio",
        "command": sys.executable,
        "args": [server_script],
    },
    "local-full": {
        "transport": "stdio",
        "command": sys.executable,
        "args": [server_full_script],
    },
})
print("=== 1. Three-Server MCP Client Configured ===")
print(f"Servers: {list(mcp_client_3.connections.keys())}\n")

# ── 2. Load tools (all servers) ────────────────────────────────────────────
tools_3 = await mcp_client_3.get_tools()
print(f"=== 2. Tools ({len(tools_3)} total) ===")
for t in tools_3:
    print(f"  - {t.name}: {t.description[:70]}")

# ── 3. Load resources (local-resources only) ──────────────────────────────
mcp_client_res = MultiServerMCPClient({
    "local-resources": {
        "transport": "stdio",
        "command": sys.executable,
        "args": [server_script],
    },
})
resources_3 = await mcp_client_res.get_resources()
print(f"\n=== 3. Resources ({len(resources_3)} total) ===")
for r in resources_3:
    print(f"  - {r.source}: {str(r.data)[:55].strip()}...")

# ── 4. Load prompts (local-full only) ─────────────────────────────────────
mcp_client_prompts = MultiServerMCPClient({
    "local-full": {
        "transport": "stdio",
        "command": sys.executable,
        "args": [server_full_script],
    },
})

prompts_list = []
try:
    try:
        prompts_list = await mcp_client_prompts.get_prompts()
        print(f"\n=== 4. Prompts ({len(prompts_list)} total) ===")
        for p in prompts_list:
            print(f"  - {p.name}: {p.description}")
    except* McpError as eg:
        for exc in eg.exceptions:
            print(f"[WARN] get_prompts() McpError: {exc}")
except AttributeError:
    print("[WARN] get_prompts() not available in this version of langchain_mcp_adapters.")
    print("       Prompts will be used as static templates below.")

# ── 4b. Render a prompt with arguments ────────────────────────────────────
# API: get_prompt(server_name, prompt_name, arguments) → list[HumanMessage | AIMessage]
prompt_question = None
try:
    try:
        rendered = await mcp_client_prompts.get_prompt(
            "local-full",        # server_name  ← required first arg
            "explain_concept",   # prompt_name
            {"concept": "MCP Prompts capability"},
        )
        if rendered:
            prompt_question = rendered[0].content
            print(f"\n=== 4b. Rendered Prompt (explain_concept) ===")
            print(str(prompt_question)[:200] + "...\n")
    except* McpError as eg:
        for exc in eg.exceptions:
            print(f"[WARN] get_prompt() McpError: {exc}")
except (AttributeError, TypeError, ValueError) as e:
    print(f"[WARN] get_prompt() error: {e}")

if not prompt_question:
    prompt_question = (
        "Explain the MCP Prompts capability as if I am a beginner developer. "
        "Start with a real-world analogy, then explain it technically, "
        "then show a short code example."
    )

# ── 5. Build comprehensive three-server agent ─────────────────────────────
system_prompt = (
    "You are an expert LangChain assistant. "
    "You have: official LangChain docs via search tools, AI concept resources, "
    "and LLM-dev utility tools (estimate_tokens, chunk_text, format_prompt).\n\n"
    "Guidelines:\n"
    "- Use search tools for documentation questions.\n"
    "- Use utility tools when the user needs token estimates, chunking, or prompt formatting.\n"
    "- Be concise and include code examples where helpful.\n"
    "- After answering, suggest a related topic to explore."
)

agent_3 = create_react_agent(model=llm, tools=tools_3, prompt=system_prompt)
print(f"\n=== 5. Agent Ready ===")
print(f"Tools: {[t.name for t in tools_3]}\n")

# ── 6. Test queries ────────────────────────────────────────────────────────
test_queries = [
    (
        "MCP prompt → agent",
        prompt_question,
    ),
    (
        "token estimation",
        "How many tokens is this text approximately: "
        "'Retrieval-Augmented Generation (RAG) enhances LLM responses by retrieving "
        "relevant context from a knowledge base before generating an answer.'? "
        "Use the estimate_tokens tool.",
    ),
    (
        "chunking",
        "Chunk this text into pieces of 80 characters with 15 overlap and show the result: "
        "'LangChain is an open-source framework for building LLM-powered applications. "
        "It provides tools for chaining prompts, managing memory, and connecting to external data.'",
    ),
    (
        "prompt formatting + docs",
        "Format this template: 'You are an expert in {domain}. Help the user understand {topic}.' "
        "with domain='LangChain' and topic='agents'. Then briefly explain what a LangChain agent is.",
    ),
]

for i, (mode, question) in enumerate(test_queries, 1):
    print(f"{'='*60}")
    print(f"Query {i} [{mode}]")
    print("="*60)
    result = await agent_3.ainvoke({"messages": [HumanMessage(content=question)]})
    print(result["messages"][-1].content)
    print()

# ── 7. Cleanup ─────────────────────────────────────────────────────────────
del mcp_client_3, mcp_client_res, mcp_client_prompts
print("=== Cleanup Complete ===")

=== 1. Three-Server MCP Client Configured ===
Servers: ['langchain-docs', 'local-resources', 'local-full']

=== 2. Tools (4 total) ===
  - SearchDocsByLangChain: Search across the Docs by LangChain knowledge base to find relevant in
  - estimate_tokens: Estimate the number of GPT tokens in a text string. Uses the ~4 chars/
  - chunk_text: Split a text into overlapping chunks by character count. Useful for RA
  - format_prompt: Fill a prompt template that uses {variable} placeholders. Pass the tem

=== 3. Resources (3 total) ===
  - None: Retrieval-Augmented Generation (RAG)
==================...
  - None: AI Agents
An agent lets an LLM decide which t...
  - None: Model Context Protocol (MCP)
==========================...
[WARN] get_prompts() not available in this version of langchain_mcp_adapters.
       Prompts will be used as static templates below.
[WARN] get_prompt() error: MultiServerMCPClient.get_prompt() takes 3 positional arguments but 4 were given

=== 5. Agent Ready ===
Tools

/var/folders/gh/r4_2cb497nl1c31dzl76npj80000gp/T/ipykernel_5549/1391192552.py:111: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_3 = create_react_agent(model=llm, tools=tools_3, prompt=system_prompt)


### Real-World Analogy

Imagine you're a chef in a restaurant. You have a collection of recipes (prompts) that guide you on how to prepare various dishes. Each recipe has specific instructions and ingredients that help you create a delicious meal. Now, if you could share these recipes with other chefs (applications) and they could use them to make their own versions of the dishes, that would be similar to how the Model Context Protocol (MCP) works with prompts. It allows different applications to access and use standardized prompts to interact with language models (LLMs) effectively.

### Technical Explanation

The Model Context Protocol (MCP) is an open standard that defines how applications can provide tools and context to LLMs. In the context of LangChain, MCP allows for the creation and management of reusable prompt templates. These templates can be stored on MCP servers and retrieved by clients (like your application) to generate messages for LLMs.

When you use MCP prompts, you c

---
##### Debugging Log — Issues Encountered in Step 5 & 6 (Revisited)

---

**Issue 1 — `SyntaxError: cannot have both 'except' and 'except*' on the same 'try'`**

**What happened:**
`except* McpError` and `except AttributeError` were in the same `try` block.

**Why:**
Python 3.11 forbids mixing `except` and `except*` in the same `try`. `except*` handles `ExceptionGroup` wrappers (what anyio produces); `AttributeError` is a plain synchronous exception and cannot be caught with `except*`.

**Fix:**
Nested `try` blocks — inner uses `except*`, outer uses plain `except`:
```python
try:
    try:
        prompts_list = await mcp_client_prompts.get_prompts()
    except* McpError as eg:      # catches ExceptionGroup wrapping McpError
        ...
except AttributeError:           # catches plain sync AttributeError
    ...
```

---

**Issue 2 — `ValueError: Couldn't find a server with name 'explain_concept'`**

**What happened:**
`await mcp_client_prompts.get_prompt("explain_concept", {...})` raised a `ValueError`.

**Why:**
The actual `langchain_mcp_adapters` signature is:
```python
get_prompt(server_name: str, prompt_name: str, arguments: dict | None = None)
```
The **first argument is the server name**, not the prompt name.

**Fix:**
```python
rendered = await mcp_client_prompts.get_prompt(
    "local-full",        # server_name  ← first arg
    "explain_concept",   # prompt_name
    {"concept": "MCP Prompts capability"},
)
```

---

**Issue 3 — `AttributeError: 'MultiServerMCPClient' has no attribute 'get_prompts'`**

**Why:** The installed version of `langchain_mcp_adapters` does not yet expose `get_prompts()`.
**Fix:** Outer `except AttributeError` handles this gracefully; prompt falls back to a static template.

---

**Issue 4 — `call_tool()` returns wrong type**

**Why:** `call_tool()` must return `list[TextContent]`, not a plain string (unlike `read_resource()`).
**Fix:** `local_mcp_server_full.py` returns `[TextContent(type='text', text=result)]`.

---

**Issue 5 — `ToolException: Input validation error: 'variables' is a required property`**

**What happened:**
The agent called `format_prompt` without a `variables` argument. The JSON schema validation rejected the call.

**Why:**
The original `variables` parameter had `"type": "object"` in the JSON schema. LLMs (including gpt-4o-mini) often struggle with nested object parameters — they tend to flatten the structure and pass `domain="LangChain"` and `topic="agents"` as top-level arguments instead of nesting them under a `variables` key.

**Fix:**
Changed `variables` from `"type": "object"` to `"type": "string"` in the tool's `inputSchema`. The description now instructs the LLM to pass a JSON string:
```
variables='{"domain": "LangChain", "topic": "agents"}'
```
The `call_tool` handler uses `json.loads()` to parse it, with a dict fallback for safety.

---
**Step 7 (Optional): Compare MCP vs Direct API Integration**

**Objective:** Understand when to use MCP vs direct API calls.

The agent, the queries, and the tool logic are **identical** in both approaches.  
The only difference is **where the tools come from**:
- **MCP approach** (Steps 5 & 6) — tools live in a separate server process, loaded at runtime via the MCP protocol
- **Direct approach** (this step) — tools are plain Python functions decorated with `@tool`, defined inline

In [17]:
from langchain_core.tools import tool
import json

# ── Direct LangChain tools — same logic as local_mcp_server_full.py ───────
# No server subprocess, no MCP client, no separate process.

@tool
def estimate_tokens(text: str) -> str:
    """Estimate the number of GPT tokens in a text string using the ~4 chars/token rule."""
    estimated = max(1, len(text) // 4)
    word_count = len(text.split())
    return (
        f"Estimated tokens: ~{estimated}\n"
        f"(Based on {len(text)} characters, {word_count} words; rule: 1 token ≈ 4 chars)"
    )

@tool
def chunk_text(text: str, chunk_size: int = 200, overlap: int = 20) -> str:
    """Split a text into overlapping chunks by character count. Useful for RAG preparation."""
    chunks = []
    start = 0
    while start < len(text):
        chunks.append(text[start:start + chunk_size])
        start += chunk_size - overlap
        if start >= len(text):
            break
    lines = [f"Total chunks: {len(chunks)}\n"]
    for i, chunk in enumerate(chunks, 1):
        lines.append(f"Chunk {i}: {chunk!r}")
    return "\n".join(lines)

@tool
def format_prompt(template: str, variables: str) -> str:
    """Fill a prompt template with {variable} placeholders. Pass variables as a JSON string."""
    try:
        return template.format_map(json.loads(variables))
    except (json.JSONDecodeError, ValueError, KeyError) as e:
        return f"Error: {e}"

direct_tools = [estimate_tokens, chunk_text, format_prompt]
print(f"Direct tools defined: {[t.name for t in direct_tools]}")
print("No MCP server, no subprocess, no client setup needed.")

Direct tools defined: ['estimate_tokens', 'chunk_text', 'format_prompt']
No MCP server, no subprocess, no client setup needed.


In [18]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

direct_agent = create_react_agent(
    model=llm,
    tools=direct_tools,
    prompt=(
        "You are an expert LangChain assistant with LLM-dev utility tools. "
        "Use estimate_tokens, chunk_text, and format_prompt when appropriate. "
        "Be concise and include code examples."
    ),
)

print("=== Direct Agent (no MCP) ===")
print(f"Tools: {[t.name for t in direct_tools]}\n")

test_queries = [
    (
        "token estimation",
        "How many tokens is this text approximately: "
        "'Retrieval-Augmented Generation (RAG) enhances LLM responses by retrieving "
        "relevant context from a knowledge base before generating an answer.'? "
        "Use the estimate_tokens tool.",
    ),
    (
        "chunking",
        "Chunk this text into pieces of 80 characters with 15 overlap: "
        "'LangChain is an open-source framework for building LLM-powered applications. "
        "It provides tools for chaining prompts, managing memory, and connecting to external data.'",
    ),
    (
        "prompt formatting",
        "Format this template: 'You are an expert in {domain}. Help the user understand {topic}.' "
        "with domain='LangChain' and topic='agents'.",
    ),
]

for i, (mode, question) in enumerate(test_queries, 1):
    print(f"{'='*60}")
    print(f"Query {i} [{mode}]")
    print("="*60)
    result = await direct_agent.ainvoke({"messages": [HumanMessage(content=question)]})
    print(result["messages"][-1].content)
    print()

/var/folders/gh/r4_2cb497nl1c31dzl76npj80000gp/T/ipykernel_5549/3567527120.py:4: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  direct_agent = create_react_agent(


=== Direct Agent (no MCP) ===
Tools: ['estimate_tokens', 'chunk_text', 'format_prompt']

Query 1 [token estimation]
The estimated number of tokens for the provided text is approximately 35.

Query 2 [chunking]
The text has been chunked into 3 pieces as follows:

**Chunk 1:**  
'LangChain is an open-source framework for building LLM-powered applications. It '

**Chunk 2:**  
'plications. It provides tools for chaining prompts, managing memory, and connect'

**Chunk 3:**  
'ry, and connecting to external data.'

Query 3 [prompt formatting]
The formatted prompt is: 

"You are an expert in LangChain. Help the user understand agents."



---
##### Key Comparisons


**MCP Integration**

| | |
|---|---|
| **Pros** | Standardized protocol — reusable across frameworks and languages |
| | Built-in abstraction — agent doesn't care where tools run |
| | Easy to add integrations — connect a new server without touching agent code |
| | Tools can be remote — HTTP servers, cloud services, external APIs |
| **Cons** | Additional abstraction layer — more moving parts |
| | Requires server setup — separate file, subprocess, or HTTP service |
| | Potential performance overhead — subprocess spawn or network round-trip per call |
| **Best for** | Multiple integrations, standardized patterns, cross-framework compatibility, tools shared across apps |

**Direct API Integration (`@tool`)**

| | |
|---|---|
| **Pros** | Direct control — no abstraction overhead |
| | Simpler for single integrations — one file, no server |
| | Better performance — tools run in-process |
| | Easier to test and debug — plain Python functions |
| **Cons** | Custom code per integration — must reimplement for each agent |
| | Not reusable — tied to one codebase and language |
| | More maintenance — tool logic lives inside the app |
| | Inconsistent interfaces — each project does it differently |
| **Best for** | Single specific integration, full API control needed, performance-critical applications, prototyping |

---

**Summary Table**

| Aspect | MCP | Direct `@tool` |
|---|---|---|
| Tool discovery | Dynamic at runtime | Hardcoded |
| Reusability | Any app, any language | Python / LangChain only |
| Process overhead | Subprocess or HTTP | None — in-process |
| Setup complexity | Server + client config | Just Python |
| Update tools | Edit server, restart | Edit notebook / module |
| Best scenario | Shared services, multi-app, cross-framework | Single-app, prototyping, speed |

---

#### Appendix — MCP Setup Reference

The following documents the complete MCP (Model Context Protocol) configuration  
completed for Claude Code on macOS, verified during this lab session.

**Configuration File Locations**

| Editor | Config File Path |
|--------|-----------------|
| Claude Code | `~/.claude/mcp-servers.json` |
| Claude Desktop | `~/Library/Application Support/Claude/claude_desktop_config.json` |

> Claude Code manages its own separate config file at `~/.claude/mcp-servers.json`.  
> Both files were created during setup. The active config for Claude Code is  
> `~/.claude/mcp-servers.json`.

**Configured Server — Filesystem**

| Property | Value |
|----------|-------|
| Package | `@modelcontextprotocol/server-filesystem` |
| Command | `npx` |
| Scope | `/Users/dinabosmabuczynska/Desktop/bootcamp_env` |
| Credentials | None required |

```json
{
  "mcpServers": {
    "filesystem": {
      "command": "npx",
      "args": [
        "-y",
        "@modelcontextprotocol/server-filesystem",
        "/Users/dinabosmabuczynska/Desktop/bootcamp_env"
      ]
    }
  }
}
```

**Filesystem Server Tools**

All tools are scoped to `bootcamp_env`. Claude cannot access files outside this path.

| Tool | Description |
|------|-------------|
| `read_file` | Read the contents of a file |
| `write_file` | Write or create a file |
| `list_directory` | List all files and folders in a directory |
| `create_directory` | Create a new folder |
| `move_file` | Move or rename a file |
| `delete_file` | Delete a file |
| `search_files` | Search for files by name pattern |
| `get_file_info` | Get metadata about a file (size, dates, etc.) |

**Tests Performed (Claude Code session)**

1. **List directory** — Claude listed `bootcamp_env` and identified all folders and `mcp_extra.ipynb`.
2. **Read file** — Claude read `mcp_test.txt` (content: `Hello MCP! This is a test file.`).
3. **Write file** — Claude created `mcp_hello.txt` with content `MCP is working!`.
4. **Query MCP tools** — Claude read `~/.claude/mcp-servers.json` and listed all 8 filesystem tools.


**Issues Encountered and Solutions**

| Issue | Cause | Solution |
|-------|-------|----------|
| Config folder did not exist | Fresh Claude Code install | `mkdir -p ~/.claude` |
| Library folder not visible in Finder | macOS hides Library by default | Terminal or Option key in Finder Go menu |
| Claude Code used different config path | Separate config from Claude Desktop | Both files accepted; Claude Code uses `~/.claude/mcp-servers.json` |
| `mcp_test.txt` not found on first attempt | File not yet created | Created via Terminal; Claude read it successfully on second attempt |

**Notes**

- MCP servers run locally via `npx` — requires Node.js (`v24.13.1` used here).
- The filesystem server is scoped to one directory — this is a security feature.
- Always restart Claude Code after editing `mcp-servers.json`.
- `mcp_hello.txt` and `mcp_test.txt` in `bootcamp_env/` remain as proof of successful read/write.